# ДЗ
Используем набор данных о винах. Вам необходимо будет предсказать качество вина по его характеристикам.

1. Для начала подготовим данные к бинарной классификации. Условно разделим вино на хорошее и плохое. Хорошим вином будем называть вино, параметр quality которого  6, плохим — всё остальное.

2. Разделим на тренировочную и тестовую, параметры test_size=0.2, random_state=42

3. Обучите на тренировочной выборке модель случайного леса, состоящего из 500 деревьев. Максимальная глубина каждого из деревьев — 8. Минимальное число объектов в листе — 10. В качестве критерия информативности используйте критерий Джини. Количество признаков, на которых обучается каждое из деревьев, возьмите как корень из m, где m — общее количество признаков. В качестве значения параметра random_state возьмите число 42.

4. Сделайте предсказание качества вина для тренировочного и тестового наборов данных.

5. Дополнительно: обучить модель логистической регрессии/попробовать применить другие ансамбли

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')
cwd = os.getcwd()
print(cwd)

Mounted at /content/drive
/content


In [53]:
df = pd.read_csv('drive/MyDrive/wine.csv', sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [55]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [56]:
df['quality'] = df['quality'].apply(lambda x: 'Good' if x == 6 else 'Bad') # "Хорошим вином будем называть вино, параметр quality которого 6, плохим — всё остальное."
df['quality'].unique()

array(['Bad', 'Good'], dtype=object)

In [57]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Bad
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,Bad
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,Bad
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,Good
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,Bad
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,Good
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,Good
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,Bad


In [58]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

X = df.drop('quality', axis=1)
y = df['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train.shape)
print(y_test.shape)

(1279,)
(320,)


In [59]:
rfc = RandomForestClassifier(500, max_depth=8, min_samples_leaf=10, random_state=42) # Модель с заданными характеристиками
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=10, n_estimators=500,
                       random_state=42)

In [60]:
y_train_predict = rfc.predict(X_train)
y_test_predict = rfc.predict(X_test)
print('Train data\n', metrics.classification_report(y_train, y_train_predict))
print('Test data\n', metrics.classification_report(y_test, y_test_predict))

Train data
               precision    recall  f1-score   support

         Bad       0.82      0.93      0.87       773
        Good       0.86      0.68      0.76       506

    accuracy                           0.83      1279
   macro avg       0.84      0.81      0.82      1279
weighted avg       0.84      0.83      0.83      1279

Test data
               precision    recall  f1-score   support

         Bad       0.64      0.80      0.71       188
        Good       0.56      0.37      0.45       132

    accuracy                           0.62       320
   macro avg       0.60      0.58      0.58       320
weighted avg       0.61      0.62      0.60       320



In [62]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train) # Переобученная модель случайного леса

RandomForestClassifier(random_state=42)

In [63]:
y_train_predict = rfc.predict(X_train)
y_test_predict = rfc.predict(X_test)
print('Train data\n', metrics.classification_report(y_train, y_train_predict))
print('Test data\n', metrics.classification_report(y_test, y_test_predict))

Train data
               precision    recall  f1-score   support

         Bad       1.00      1.00      1.00       773
        Good       1.00      1.00      1.00       506

    accuracy                           1.00      1279
   macro avg       1.00      1.00      1.00      1279
weighted avg       1.00      1.00      1.00      1279

Test data
               precision    recall  f1-score   support

         Bad       0.73      0.79      0.76       188
        Good       0.66      0.58      0.62       132

    accuracy                           0.71       320
   macro avg       0.70      0.69      0.69       320
weighted avg       0.70      0.71      0.70       320



In [64]:
from sklearn import linear_model
log_reg = linear_model.LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train) # Модель логистической регрессии

LogisticRegression(max_iter=1000, random_state=42)

In [66]:
y_train_predict = log_reg.predict(X_train)
y_test_predict = log_reg.predict(X_test)
print('Train data\n', metrics.classification_report(y_train, y_train_predict))
print('Test data\n', metrics.classification_report(y_test, y_test_predict))

Train data
               precision    recall  f1-score   support

         Bad       0.63      0.83      0.72       773
        Good       0.51      0.27      0.35       506

    accuracy                           0.61      1279
   macro avg       0.57      0.55      0.54      1279
weighted avg       0.58      0.61      0.57      1279

Test data
               precision    recall  f1-score   support

         Bad       0.61      0.79      0.69       188
        Good       0.48      0.28      0.35       132

    accuracy                           0.58       320
   macro avg       0.54      0.53      0.52       320
weighted avg       0.56      0.58      0.55       320



In [67]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(max_depth=3, random_state=42)
ada = AdaBoostClassifier(estimator=dtc, random_state=42)
ada.fit(X_train, y_train) # Модель адаптивного бустинга

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3,
                                                    random_state=42),
                   random_state=42)

In [68]:
y_train_predict = ada.predict(X_train)
y_test_predict = ada.predict(X_test)
print('Train data\n', metrics.classification_report(y_train, y_train_predict))
print('Test data\n', metrics.classification_report(y_test, y_test_predict))

Train data
               precision    recall  f1-score   support

         Bad       0.97      0.98      0.98       773
        Good       0.97      0.96      0.96       506

    accuracy                           0.97      1279
   macro avg       0.97      0.97      0.97      1279
weighted avg       0.97      0.97      0.97      1279

Test data
               precision    recall  f1-score   support

         Bad       0.71      0.77      0.74       188
        Good       0.62      0.55      0.59       132

    accuracy                           0.68       320
   macro avg       0.67      0.66      0.66       320
weighted avg       0.67      0.68      0.67       320

